Notebook that pulls pollution data from the City of Montreal Open Data Portal, cleans it, and presents it in charts. 

In [2]:
# import libraires
import requests, json, glob
import plotly.express as px
import pandas as pd

In [3]:
# I created a dictionary with names for the years and the api URLS. I hope these are still valid if anyone tries this!
years={"seventeen": 'https://montreal.l3.ckan.io/api/3/action/datastore_search?resource_id=c07d9a0e-534f-4663-b0d5-3e5ce594834f&sort=_id asc&limit=1000000', 
"eighteen": 'https://montreal.l3.ckan.io/api/3/action/datastore_search?resource_id=f2667b58-71da-4cec-a8ff-a45f028cd831&sort=_id asc&limit=1000000',
 "nineteen":'https://montreal.l3.ckan.io/api/3/action/datastore_search?resource_id=68da6084-de06-4514-9371-754c81453f9a&sort=_id asc&limit=1000000', 
 "twenty":'https://montreal.l3.ckan.io/api/3/action/datastore_search?resource_id=811e76d0-90cb-4703-bd6f-bd0faff81f4a&sort=_id asc&limit=1000000',
  "twentyone":'https://montreal.l3.ckan.io/api/3/action/datastore_search?resource_id=ca30e082-90ad-4ce2-806c-7f78d3be74c9&sort=_id asc&limit=1000000'}
swap = {'m p-xylene': "mpxylene", 'o-xylene': "oxylene", "date_ heure": "date", "date_heure": "date"}


In [4]:
for i in years: 
    df = pd.DataFrame(json.loads(requests.get(years[i]).text)['result']['records'])
    df = df.loc[:, ~df.columns.isin(['Unnamed: 0','_id',])]
    columns = [x.lower() for x in df.columns]
    for j in columns: 
        for k in swap.keys():
            if k == j: 
                columns[columns.index(k)] = swap[k]
    df.columns = columns
    if i == 'twenty': 
        df['date']=df['date'].str.split(" ", n = 1, expand = True)[0]
    df.to_csv('ground_pollution/'+str(i)+".csv", index=None) 

In [ ]:
path = "ground_pollution/*.csv"
for i in enumerate(glob.glob(path)): 
    if i[0] == 0 and i[1] != 'ground_pollution/combined.csv':
        main=pd.read_csv(os.path.abspath(i[1]), low_memory=False)
    else: 
        df=pd.read_csv(os.path.abspath(i[1]), low_memory=False)
        main=main.append(df, ignore_index = True)
        main.to_csv('ground_pollution/'+"combined"+".csv", index=None)

In [ ]:
df=pd.read_csv('/Users/cassidymacdonald/Documents/g_465_final_proj/ground_pollution/combined.csv', low_memory=False)
df=df[pd.to_numeric(df['so2'], errors='coerce').notnull()]
df[['so2']] = df[['so2']].astype("float")
#df=df[pd.to_numeric(df['pm2_5'], errors='coerce').notnull()]
#df[['pm2_5']] = df[['pm2_5']].astype("float")
df['Date']= pd.to_datetime(df['date'])
df=df.groupby(pd.Grouper(key='Date',freq='Q')).mean()


NameError: name 'pd' is not defined

In [ ]:
fig = px.line(df, x=df.index, y="so2")
fig.show()